In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,7087
2,Huelva,Confirmados PDIA 14 días,2580
3,Huelva,Tasa PDIA 14 días,"505,1464239353256"
4,Huelva,Confirmados PDIA 7 días,1285
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,7
629,Municipio de Huelva sin especificar,Total Confirmados,102
630,Municipio de Huelva sin especificar,Curados,22


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  7087.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1931.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 409 personas en los últimos 7 días 

Un positivo PCR cada 195 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,7087.0,1285.0,2580.0,510743.0,251.594246,505.146424,289.0
Huelva-Costa,3697.0,740.0,1431.0,288115.0,256.841886,496.676674,160.0
Condado-Campiña,2221.0,365.0,725.0,155057.0,235.397306,467.569990,91.0
Huelva (capital),1931.0,351.0,738.0,143663.0,244.321781,513.702206,68.0
Sierra de Huelva-Andévalo Central,1068.0,173.0,395.0,67571.0,256.026994,584.570304,46.0
Lepe,578.0,133.0,242.0,27431.0,484.852904,882.213554,27.0
Almonte,338.0,42.0,94.0,24191.0,173.618288,388.574263,26.0
Gibraleón,308.0,32.0,115.0,12607.0,253.827239,912.191640,19.0
Isla Cristina,201.0,45.0,68.0,21264.0,211.625282,319.789315,16.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Aracena,269.0,57.0,159.0,8107.0,703.096090,1961.268040,14.0
Almendro (El),15.0,7.0,9.0,826.0,847.457627,1089.588378,4.0
Villablanca,39.0,24.0,28.0,2848.0,842.696629,983.146067,2.0
Palma del Condado (La),316.0,51.0,105.0,10761.0,473.933649,975.745749,5.0
Gibraleón,308.0,32.0,115.0,12607.0,253.827239,912.191640,19.0
Lepe,578.0,133.0,242.0,27431.0,484.852904,882.213554,27.0
Valverde del Camino,233.0,48.0,103.0,12820.0,374.414977,803.432137,14.0
San Silvestre de Guzmán,6.0,1.0,5.0,623.0,160.513644,802.568218,1.0
Escacena del Campo,85.0,4.0,18.0,2284.0,175.131349,788.091068,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,34.0,1.0,7.0,2791.0,35.829452,250.806163,0.0,0.142857
Cumbres Mayores,10.0,1.0,5.0,1755.0,56.980057,284.900285,0.0,0.200000
Higuera de la Sierra,13.0,1.0,5.0,1287.0,77.700078,388.500389,1.0,0.200000
Encinasola,7.0,1.0,5.0,1305.0,76.628352,383.141762,0.0,0.200000
San Silvestre de Guzmán,6.0,1.0,5.0,623.0,160.513644,802.568218,1.0,0.200000
Escacena del Campo,85.0,4.0,18.0,2284.0,175.131349,788.091068,0.0,0.222222
Gibraleón,308.0,32.0,115.0,12607.0,253.827239,912.191640,19.0,0.278261
Cortegana,124.0,2.0,7.0,4666.0,42.863266,150.021432,1.0,0.285714
Manzanilla,43.0,4.0,12.0,2135.0,187.353630,562.060890,0.0,0.333333
